In [1]:
%matplotlib inline

In [2]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import lightkurve
import exoplanet as xo
import pymc3 as pm
import theano.tensor as tt

ModuleNotFoundError: No module named 'exoplanet'

In [ ]:
target = 'AU Mic'
lcf = lightkurve.search_lightcurvefile(target).download()
lc = lcf.get_lightcurve('PDCSAP_FLUX').normalize()

In [ ]:
lc.flux = (lc.flux - 1.0) * 1e3

In [ ]:
lc.plot(normalize=False)

In [ ]:
lc_hdr = lcf.header(ext=1)
texp = lc_hdr["FRAMETIM"] * lc_hdr["NUM_FRM"]
texp /= 60.0 * 60.0 * 24.0
time = lc.time
flux = lc.flux
flux_err = lc.flux_err
m = np.isfinite(time) & np.isfinite(flux)
time = time[m]
flux = flux[m]
flux_err = flux_err[m]

In [ ]:
# Identify outliers
m = np.ones(len(flux), dtype=bool)
for i in range(10):
    y_prime = np.interp(time, time[m], flux[m])
    smooth = savgol_filter(y_prime, 501, polyorder=3)
    resid = flux - smooth
    sigma = np.sqrt(np.mean(resid**2))
    m0 = resid < sigma*0.5
    if m.sum() == m0.sum():
        m = m0
        break
    m = m0

In [ ]:
lc.plot(normalize=False)
plt.plot(time[m], flux[m], lw=1)
plt.plot(time, smooth, lw=1)
plt.xlim(1330,1332)

In [ ]:
ref_time = 0.5 * (np.min(time[m])+np.max(time[m]))
time = np.ascontiguousarray(time[m] - ref_time, dtype=np.float64)
flux = np.ascontiguousarray(flux[m], dtype=np.float64)
flux_err = np.ascontiguousarray(flux_err[m], dtype=np.float64)

x = time
y = flux
yerr = flux_err

In [ ]:
plt.plot(time, flux, ".k")
plt.plot(time, smooth[m])
# plt.xlim(1342.23-0.4-ref_time,1342.23+0.4-ref_time)


In [ ]:
results = xo.estimators.lomb_scargle_estimator(
    x, y, max_peaks=1, min_period=1.0, max_period=30.0,
    samples_per_peak=50)

peak = results["peaks"][0]
ls_period = peak["period"]
freq, power = results["periodogram"]
plt.plot(-np.log10(freq), power, "k")
plt.axvline(np.log10(ls_period), color="k", lw=4, alpha=0.3)
plt.xlim((-np.log10(freq)).min(), (-np.log10(freq)).max())
plt.annotate("period = {0:.4f} d".format(ls_period),
             (0, 1), xycoords="axes fraction",
             xytext=(5, -5), textcoords="offset points",
             va="top", ha="left", fontsize=12)
plt.yticks([])
plt.xlabel("log10(period)")
plt.ylabel("power");

In [ ]:
def build_model(mask=None):
    p_period = 16.93, 30.537
    p_t0 = -8.84, 2.9955
    p_depth = 0.04, 0.03
    if mask is None:
        mask = np.ones_like(x, dtype=bool)
    with pm.Model() as model:

        # The mean flux of the time series
        mean = pm.Normal("mean", mu=6, sd=15.0)

        # A jitter term describing excess white noise
        logs2 = pm.Normal("logs2", mu=2*np.log(np.min(yerr[mask])), sd=10.0,)

        # A SHO term to capture long term trends
        logS = pm.Normal("logS", mu=0.0, sd=15.0, testval=np.log(np.var(y[mask])))
        logw = pm.Normal("logw", mu=np.log(2*np.pi/10.0), sd=10.0)
        term1 = xo.gp.terms.SHOTerm(log_S0=logS, log_w0=logw, Q=1/np.sqrt(2))
        
        # The parameters of the RotationTerm kernel
        logamp = pm.Normal("logamp", mu=np.log(np.var(y[mask])), sd=5.0)
        logperiod = pm.Normal("logperiod", mu=np.log(ls_period), sd=5.0)
        period = pm.Deterministic("period", tt.exp(logperiod))
        logQ0 = pm.Normal("logQ0", mu=1.0, sd=10.0)
        logdeltaQ = pm.Normal("logdeltaQ", mu=2.0, sd=10.0)
        mix = pm.Uniform("mix", lower=0, upper=1.0)
        term2 = xo.gp.terms.RotationTerm(
            log_amp=logamp,
            period=period,
            log_Q0=logQ0,
            log_deltaQ=logdeltaQ,
            mix=mix
        )

        u_star = xo.distributions.QuadLimbDark("u_star",
                                              testval=np.array([0.4, 0.2]))
        R_star = 0.75, 0.1
        Rho_star = 1
        r_star = pm.Normal("r_star", mu=R_star[0], sd=R_star[1])
        logrho_star = pm.Normal("logrho_star", mu=np.log(Rho_star), sd=1)
        rho_star = pm.Deterministic("rho_star", tt.exp(logrho_star))
        pm.Potential("r_star_prior", tt.switch(r_star > 0, 0, -np.inf))
        logP = pm.Normal("logP", mu=np.log(p_period), sd=0.01, shape=2, testval=np.log(p_period))
        t0 = pm.Normal("t0", mu=p_t0, sd=0.1, shape=2, testval=p_t0)
#         ror, b = xo.distributions.get_joint_radius_impact(
#             min_radius=0.001, max_radius=0.3,
#             testval_r=p_depth,
#             testval_b=0.1)
        logror = pm.Normal("logror", mu=np.log(p_depth),
                           sd=2, shape=2)
        ror = pm.Deterministic("ror", tt.exp(logror))
        b_param = pm.Uniform("b_param", lower=0, upper=1, shape=2, testval=[0.2, 0.4])
        b = pm.Deterministic("b", b_param * (1 + ror))
        
        ecc = pm.Bound(pm.Beta, lower=0.0, upper=1.0)("ecc", alpha=0.867, beta=3.03, testval=0.1,
                                                     shape=2)
        omega = xo.distributions.Angle("omega", shape=2)
        
#         ecc= [0., 0.]
#         omega = [0, 0]

        
        pm.Potential("ror_prior_lo", tt.switch(tt.all(0.001 < ror), 0.0, -np.inf))
        pm.Potential("ror_prior_hi", tt.switch(tt.all(ror < 0.3), 0.0, -np.inf))  
#         pm.Potential("ror_prior", -tt.log(ror))

    #         pm.Potential("b_prior",  tt.switch(b < 1, 0, -np.inf))
        p_period = pm.Deterministic("p_period", tt.exp(logP))
        r_pl = pm.Deterministic("r_pl", r_star * ror)
        r_ple = pm.Deterministic("r_ple", r_star * ror / 0.009155)
        orbit = xo.orbits.KeplerianOrbit(
            r_star=r_star,
            period=p_period, t0=t0, b=b,
            rho_star=rho_star, ecc=ecc, omega=omega)
        light_curves = xo.StarryLightCurve(u_star).get_light_curve(
            orbit=orbit, r=r_pl, t=x[mask], texp=texp)*1e3
        light_curve = pm.math.sum(light_curves, axis=-1)
        pm.Deterministic("light_curves", light_curves)

        # Set up the Gaussian Process model
        kernel = term1 + term2
        gp = xo.gp.GP(kernel, x[mask], yerr[mask]**2 + tt.exp(logs2), J=6)

        # Compute the Gaussian Process likelihood and add it into the
        # the PyMC3 model as a "potential"
        pm.Potential("loglike", gp.log_likelihood(y[mask] - mean - light_curve))

        # Compute the mean model prediction for plotting purposes
        pm.Deterministic("pred", gp.predict())

        # Optimize to find the maximum a posteriori parameters
        map_soln = pm.find_MAP(start=model.test_point, vars=[mean, logs2])
        map_soln = pm.find_MAP(start=map_soln, vars=[mean, logs2, logS, logw])
        map_soln = pm.find_MAP(start=map_soln, vars=[mean, logs2, logamp, logQ0, logdeltaQ, mix])
#         map_soln = pm.find_MAP(start=map_soln, vars=[model.logror, model.b_param])
        map_soln = pm.find_MAP(start=map_soln, vars=[model.logror, model.b_param, logP, t0])
#         map_soln = pm.find_MAP(start=map_soln,)
    return model, map_soln

model0, map_soln0 = build_model()

In [ ]:
map_soln0

In [ ]:
mod = map_soln0["pred"] + map_soln0["mean"] + np.sum(map_soln0["light_curves"], axis=-1)
resid = y - mod
rms = np.sqrt(np.median(resid**2))
mask = np.abs(resid) < 7. * rms

fig, axes = plt.subplots(4, 1, figsize=(10, 10))
ax1 = axes[0]
ax1.plot(x, resid, "k", label="data")
ax1.plot(x, np.sum(map_soln0["light_curves"], axis=-1))
ax1.plot(x[~mask], resid[~mask], "xr", label="outliers")
ax1.axhline(0, color="#aaaaaa", lw=1)
ax1.set_ylabel("residuals [ppt]")
ax1.set_xlabel("time [days]")
ax1.legend(fontsize=12, loc=4)
ax1.set_xlim(x.min(), x.max());
ax1.set_xlim(-9.5,-8)

ax2 = axes[1]
ax2.plot(x, y, "k", label="data")
ax2.plot(x, np.sum(map_soln0["light_curves"], axis=-1))
ax2.plot(x[~mask], y[~mask], "xr", label="outliers")
ax2.axhline(0, color="#aaaaaa", lw=1)
ax2.set_ylabel("residuals [ppt]")
ax2.set_xlabel("time [days]")
ax2.legend(fontsize=12, loc=4)
ax2.set_xlim(x.min(), x.max());
ax2.set_xlim(-9.5,-8)


ax3 = axes[2]
ax3.plot(x, resid, "k", label="data")
ax3.plot(x, np.sum(map_soln0["light_curves"], axis=-1))
ax3.plot(x[~mask], resid[~mask], "xr", label="outliers")
ax3.axhline(0, color="#aaaaaa", lw=1)
ax3.set_ylabel("residuals [ppt]")
ax3.set_xlabel("time [days]")
ax3.legend(fontsize=12, loc=4)
ax3.set_xlim(x.min(), x.max());
ax3.set_xlim(-9.5+12,-8+12)

ax4 = axes[3]
ax4.plot(x, y, "k", label="data")
ax4.plot(x, np.sum(map_soln0["light_curves"], axis=-1))
ax4.plot(x[~mask], y[~mask], "xr", label="outliers")
ax4.axhline(0, color="#aaaaaa", lw=1)
ax4.set_ylabel("residuals [ppt]")
ax4.set_xlabel("time [days]")
ax4.legend(fontsize=12, loc=4)
ax4.set_xlim(x.min(), x.max());
ax4.set_xlim(-9.5+12,-8+12)

In [ ]:
model, map_soln = build_model(mask)

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(6, 10))
ax1 = axes[0]
ax1.plot(x[mask], y[mask], "k", label="data")
ax1.plot(x[mask], map_soln["pred"] + map_soln["mean"], color="C1", label="model")
ax1.plot(x, np.sum(map_soln0["light_curves"], axis=-1))
ax1.legend(fontsize=10)
ax1.set_xlabel("time [days]")
ax1.set_ylabel("relative flux")
ax1.set_title("map model")
ax1.set_xlim(-9.5,-8)

ax2 = axes[1]
ax2.plot(x[mask], y[mask], "k", label="data")
ax2.plot(x[mask], map_soln["pred"] + map_soln["mean"], color="C1", label="model")
ax2.plot(x, np.sum(map_soln0["light_curves"], axis=-1))
ax2.legend(fontsize=10)
ax2.set_xlabel("time [days]")
ax2.set_ylabel("relative flux")
ax2.set_title("map model")
ax2.set_xlim(-9.5+17,-8+17)

ax3 = axes[2]
ax3.plot(x[mask], y[mask], "k", label="data")
ax3.plot(x[mask], map_soln["pred"] + map_soln["mean"], color="C1", label="model")
ax3.plot(x, np.sum(map_soln0["light_curves"], axis=-1))
ax3.legend(fontsize=10)
ax3.set_xlabel("time [days]")
ax3.set_ylabel("relative flux")
ax3.set_title("map model")
ax3.set_xlim(-9.5+12,-8+12)

In [23]:
sampler = xo.PyMC3Sampler(window=200, start=500, finish=500)
with model:
    sampler.tune(tune=3000, start=map_soln, step_kwargs=dict(target_accept=0.9))

/home/tom/anaconda3/lib/python3.6/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Sampling 4 chains:  13%|█▎        | 255/2008 [06:06<55:57,  1.92s/draws]  


ValueError: Not enough samples to build a trace.

In [135]:
with model:
#     db = pm.backends.Text('trace-out.txt')
    trace = sampler.sample(draws=2000, )#trace=db)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b_param, logror, t0, logP, logrho_star, r_star, u_star, mix, logdeltaQ, logQ0, logperiod, logamp, logw, logS, logs2, mean]
/home/tom/anaconda3/lib/python3.6/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Sampling 4 chains: 100%|██████████| 8000/8000 [03:45<00:00, 13.07draws/s]
There were 1623 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.004766948125559143, but should be close to 0.9. Try to increase the number of tuning steps.
There were 1724 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does

In [138]:
pm.summary(trace, varnames= ['b_param', 'logror', 't0', 'logP', 
           'logrho_star', 'r_star', 'u_star', 'mix', 'logdeltaQ', 'logQ0', 'logperiod', 'logamp', 'logw', 'logS',
                             'logs2', 'mean'])
                             

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
b_param__0,0.238614,0.101416,0.010096,0.102278,0.383467,NaN,2.981283
b_param__1,0.554459,0.380609,0.038045,0.153241,0.974324,2.012365,14.779869
logror__0,-2.955855,0.023095,0.002290,-3.002028,-2.913983,2.603271,2.301713
logror__1,-3.437752,0.873050,0.087174,-4.507068,-1.787041,2.105757,4.942315
t0__0,-8.843581,0.002234,0.000223,-8.847050,-8.840858,NaN,4.001116
t0__1,3.023583,0.060276,0.006021,2.971038,3.133484,2.049293,7.941368
logP__0,2.894662,0.023259,0.002318,2.859580,2.922614,2.205467,3.737579
logP__1,5.587001,1.410152,0.140700,3.311913,7.452685,2.080583,5.776305
logrho_star,1.279535,0.051667,0.005073,1.219989,1.388245,6.774003,1.333677
r_star,0.749463,0.034705,0.003422,0.709561,0.815720,2.464778,2.539178


In [24]:
with model:
#     db = pm.backends.Text('trace-out.txt')
    trace = sampler.sample(draws=2000, )#trace=db)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [omega, ecc, b_param, logror, t0, logP, logrho_star, r_star, u_star, mix, logdeltaQ, logQ0, logperiod, logamp, logw, logS, logs2, mean]
/home/tom/anaconda3/lib/python3.6/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Sampling 4 chains: 100%|██████████| 8000/8000 [1:12:03<00:00,  2.69s/draws]
There were 871 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.0014984669303228562, but should be close to 0.9. Try to increase the number of tuning steps.
There were 599 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance pro

In [41]:
xo.get_samples_from_trace(trace, size=1)

<generator object get_samples_from_trace at 0x7f655913f2b0>

In [32]:
np.log(2*np.pi)

1.8378770664093453

In [33]:
1. / np.exp(4.5)

0.011108996538242306

In [142]:
trace['p_period']

array([[ 18.55002512, 283.24401855],
       [ 18.55002512, 283.24401855],
       [ 18.55002512, 283.24401855],
       ...,
       [ 17.79828059,  27.43757372],
       [ 17.79828059,  27.43757372],
       [ 17.79828059,  27.43757372]])